<a href="https://colab.research.google.com/github/SmitaPaul7000/CaseStudies/blob/master/0_1Image_Cap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np

In [0]:
for i in range(0,5):
  print(i)

0
1
2
3
4
